In [1]:
import cv2
import time
import random
import mediapipe as mp

mp_hands= mp.solutions.hands
hands= mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing= mp.solutions.drawing_utils
E="easy"
M="medium"
H="hard"
difficulty=input("Choose a level- Easy(E), medium(M), Hard(H)")
if difficulty == "easy":
    radius = 100
    change_interval = 5
elif difficulty == "medium":
    radius = 60
    change_interval = 3
elif difficulty == "hard":
    radius = 35
    change_interval = 1.5

point_color = (0, 255, 0)
score = 0
last_position=None
positions = [(160, 300), (320, 300), (480, 300), (160, 100), (320, 100), (480, 100)]
total_attempt= 0
game_over= False
max_score= 20

def new_round():
    global last_position
    index1 = random.randint(0, len(positions)-2)
    a_position= [p for p in positions if p != last_position]
    new_position= a_position[index1]
    last_position= new_position
    print(new_position)
    return new_position

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Camera not accessible")
    exit()

r_position = new_round()
round_start_time = time.time()



while cap.isOpened():
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    # Change circle position every 4 seconds
    if time.time() - round_start_time > change_interval and not game_over:
        r_position = new_round()
        round_start_time = time.time()
        total_attempt += 1


    # Draw the target circle
    cv2.circle(frame, r_position, radius, point_color, -1)

    try:
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Index finger tip is landmark 8
                index_tip = hand_landmarks.landmark[8]
                h, w, _ = frame.shape
                idx = int(index_tip.x * w)
                idy = int(index_tip.y * h)

                # Draw fingertip
                cv2.circle(frame, (idx, idy), 10, (0, 255, 0), -1)

                # Game logic
                cx, cy = r_position
                distance = ((idx - cx) ** 2 + (idy - cy) ** 2) ** 0.5

                if distance < radius and not game_over:
                    score += 1
                    r_position = new_round()
                    round_start_time = time.time() # reset timer after hit
                    total_attempt += 1

                if score>= max_score:
                    game_over=True
                    accuracy = (score / total_attempt) * 100

            if game_over:
                cv2.putText(frame, "Game Over!", (120, 200),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)
                cv2.putText(frame, f"Accuracy: {accuracy:.2f}%", (120, 250),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 0), 2)
            else:
                    # show ongoing score
                cv2.putText(frame, f"Score: {score}", (20, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 2)
                    

    except Exception as e:
        print("Error:", e)
        pass

    # Always draw landmarks
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Display score
    cv2.putText(frame, f"Score: {score}/20", (20, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 2)
        
        

    cv2.imshow('Colorgame', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Choose a level- Easy(E), medium(M), Hard(H) easy


(480, 300)
(160, 300)
(160, 100)
(320, 300)
(480, 300)
(320, 100)
(160, 100)
(320, 300)
(480, 300)
(160, 100)
(320, 100)
(160, 100)
(160, 300)
(160, 100)
(160, 300)
(320, 100)
(320, 300)
(320, 100)
(480, 100)
(160, 300)
(320, 300)
(160, 300)


In [1]:
pip install random

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement random (from versions: none)
ERROR: No matching distribution found for random
